In [1]:
import sys
sys.path.append( '..' )
from utilities.get_data import get_historical_from_path
from os import listdir
from os.path import isfile, join
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ta
import os
import subprocess
import importlib

# Use the terminal code to import the data from the internet node .\database\sdl_for_quick_analysis.js

In [12]:
def execute_terminal_command(command):
    try:
        result = subprocess.run(command, shell=True, capture_output=True, text=True)
        # Print the standard output of the command
        print(result.stdout)

        # If you want to capture the standard error as well, uncomment the line below
        # print(result.stderr)

        # Check the return code
        if result.returncode == 0:
            print("Command executed successfully.")
        else:
            print(f"Command execution failed with return code: {result.returncode}")
    except subprocess.CalledProcessError as e:
        print(f"Error executing command: {e}")
def install_package_if_needed(package):
    try:
        # Check if the package is installed
        subprocess.run(['node', '-e', 'require.resolve("{}")'.format(package)], check=True, capture_output=True)
        print(f"{package} is already installed.")
    except subprocess.CalledProcessError as e:
        # Package not found, install it
        print(f"{package} is not installed. Installing...")
        try:
            command = f'npm install {package}'
            execute_terminal_command(command)
            print(f"{package} installed successfully.")
        except subprocess.CalledProcessError as e:
            print(f"Error installing {package}: {e}")
def remove_non_csv_files(directory):
    for filename in os.listdir(directory):
        if not filename.endswith(".csv"):
            file_path = os.path.join(directory, filename)
            if os.path.isfile(file_path):
                os.remove(file_path)
                print(f"Removed file: {file_path}")
def get_analyisis_from_window(window=24):
    metric_list = {}
    for coin in df_list:
        try:
            df = df_list[coin]
            df.drop(columns=df.columns.difference(['open','high','low','close','volume']), inplace=True)
            df["rvolume"] = df["volume"] * df["close"]
            df["olhc"] = (df["open"] + df["low"] + df["high"] + df["close"])/4
            df["atr"] = (ta.volatility.average_true_range(high=df['high'], low=df['low'], close=df['close'])/df["close"])*100
            # df["norm"].dropna(inplace=True)
            volatility = df["atr"].mean()
            mean_volume = df["rvolume"].mean()
            std = df["olhc"].std()/df["olhc"].mean()
            perf = (df["close"].iloc[-1] - df["close"].iloc[0]) / df["close"].iloc[0]
            return_vs_vol = perf/volatility
            last_volume = df["rvolume"].iloc[-window:].mean()
            volume_evolution = last_volume / mean_volume
            last_volatility = df.iloc[-window:]["olhc"].std() / df.iloc[-window:]["olhc"].mean()
            last_perf = (df.iloc[-1]["close"] - df.iloc[-window]["close"]) / df.iloc[-window]["close"]
            volatility_evolution = last_volatility / volatility
            last_return_vs_vol = last_perf / last_volatility
            metric_list[coin] = {
                "mean_volume": round(mean_volume),
                "last_volume": round(last_volume),
                "volume_evolution": round(volume_evolution,2),
                "mean_volatility": round(volatility,2),
                "last_volatility": round(last_volatility*100,2),
                "volatility_evolution": round(volatility_evolution*100,2),
                "return": round(perf*100,2),
                "last_return":round(last_perf*100,2),
                "return_vs_vol": round(return_vs_vol*100,2),
                "last_return_vs_vol":round(last_return_vs_vol,2)
                }
        except:
            # print("Error on", coin)
            # raise
            pass

    return pd.DataFrame.from_dict(metric_list, orient='index')

In [5]:
install_package_if_needed("ccxt")

ccxt is already installed.


In [6]:
install_package_if_needed("csv-writer")

csv-writer is already installed.


In [7]:
command = 'node .\\database\\dl_for_quick_analysis.js'
execute_terminal_command(command)

database/quick_analysis is deleted!

Loading 1/590 coins downloaded
Loading 2/590 coins downloaded
Loading 3/590 coins downloaded
Loading 4/590 coins downloaded
Loading 5/590 coins downloaded
Loading 6/590 coins downloaded
Loading 7/590 coins downloaded
Loading 8/590 coins downloaded
Loading 9/590 coins downloaded
Loading 10/590 coins downloaded
Loading 11/590 coins downloaded
Loading 12/590 coins downloaded
Loading 13/590 coins downloaded
Loading 14/590 coins downloaded
Loading 15/590 coins downloaded
Loading 16/590 coins downloaded
Loading 17/590 coins downloaded
Loading 18/590 coins downloaded
Loading 19/590 coins downloaded
Loading 20/590 coins downloaded
Loading 21/590 coins downloaded
Loading 22/590 coins downloaded
Loading 23/590 coins downloaded
Loading 24/590 coins downloaded
Loading 25/590 coins downloaded
Loading 26/590 coins downloaded
Loading 27/590 coins downloaded
Loading 28/590 coins downloaded
Loading 29/590 coins downloaded
Loading 30/590 coins downloaded
Loading 31/5

In [8]:
directory_path = "./database/quick_analysis"
remove_non_csv_files(directory_path)

Removed file: ./database/quick_analysis\1000FLOKI-USDT
Removed file: ./database/quick_analysis\1000LUNC-USDT
Removed file: ./database/quick_analysis\1000PEPE-USDT
Removed file: ./database/quick_analysis\1000SHIB-USDT
Removed file: ./database/quick_analysis\1000XEC-USDT
Removed file: ./database/quick_analysis\1INCH-USDT
Removed file: ./database/quick_analysis\AAVE-USDT
Removed file: ./database/quick_analysis\ACH-USDT
Removed file: ./database/quick_analysis\ADA-USDT
Removed file: ./database/quick_analysis\AGIX-USDT
Removed file: ./database/quick_analysis\ALGO-USDT
Removed file: ./database/quick_analysis\ALICE-USDT
Removed file: ./database/quick_analysis\ALPHA-USDT
Removed file: ./database/quick_analysis\AMB-USDT
Removed file: ./database/quick_analysis\ANKR-USDT
Removed file: ./database/quick_analysis\ANT-USDT
Removed file: ./database/quick_analysis\APE-USDT
Removed file: ./database/quick_analysis\API3-USDT
Removed file: ./database/quick_analysis\APT-USDT
Removed file: ./database/quick_an

In [9]:
mypath = "./database/quick_analysis"
file_list = [f for f in listdir(mypath) if isfile(join(mypath, f))]
df_list = {}
for file in file_list:
    df_list[file[:-9]] = get_historical_from_path(mypath+"/"+file)

print("All data loaded")

All data loaded


In [11]:
df_list["BTC"]

,open,high,low,close,volume
date,,,,,
2023-04-23 02:00:00,27551.14,27600.00,27483.68,27592.02,1293.26362
2023-04-23 03:00:00,27592.02,27643.57,27533.76,27557.84,1257.64328
2023-04-23 04:00:00,27557.83,27627.44,27547.04,27605.82,819.73270
2023-04-23 05:00:00,27605.82,27620.18,27531.85,27585.14,823.39319
2023-04-23 06:00:00,27585.13,27624.95,27555.70,27588.14,824.30123
...,...,...,...,...,...
2023-06-03 12:00:00,27129.78,27173.73,27129.78,27165.81,479.78328
2023-06-03 13:00:00,27165.81,27183.99,27149.88,27166.98,520.87423
2023-06-03 14:00:00,27166.97,27193.62,27166.97,27171.24,439.74420


In [13]:
df_metric = get_analyisis_from_window(12).sort_values(by="last_return", ascending=False)
df_metric.iloc[:30]

,mean_volume,last_volume,volume_evolution,mean_volatility,last_volatility,volatility_evolution,return,last_return,return_vs_vol,last_return_vs_vol
BTS,31769,954005,30.03,1.19,12.03,10.10,3.50,22.22,2.94,1.85
LEVER,97723,1548801,15.85,1.15,3.36,2.91,0.76,13.74,0.66,4.09
TRU,112883,409947,3.63,1.40,2.29,1.64,-19.22,9.17,-13.77,4.00
JST,40536,1178410,29.07,0.57,3.50,6.17,3.25,9.04,5.72,2.58
SXP,330147,1328382,4.02,1.22,3.48,2.85,-24.17,7.64,-19.77,2.19
AKRO,37574,30871,0.82,1.17,1.43,1.23,-9.13,4.78,-7.84,3.35
FLUX,83387,320590,3.84,1.31,2.82,2.15,-11.74,4.77,-8.95,1.69
MTL,185577,572996,3.09,1.15,1.58,1.38,-8.42,4.52,-7.34,2.86
OAX,150527,149744,0.99,1.77,1.80,1.02,-25.37,4.05,-14.33,2.25
DEXE,18621,24986,1.34,0.91,1.49,1.63,-15.74,4.05,-17.25,2.73
